# Cartpole

## A2C Agent 

In [ ]:
import gym
from a2c import A2CAgent 
import time
import numpy as np

# Create Gym environment
a2c_env = "CartPole-v1"
env = gym.make(a2c_env)

# Check agent class for initialization parameters and initialize agent
if a2c_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-3

agent = A2CAgent(env, gamma, lr)

# Define training parameters
max_episodes = 500
max_steps = 500

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    trajectory = []
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)
        trajectory.append((state, action, reward, next_state, done))
        episode_reward += reward  
        if done or step == max_steps:
            episode_rewards.append(episode_reward)
            print("Episode " + str(episode) + ": " + str(episode_reward))
            break
        state = next_state
    agent.update(trajectory, 0)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
a2c_rewards = episode_rewards
a2c_runtime = run_time

In [ ]:
name = './log_files/a2c/' + a2c_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((a2c_runtime, a2c_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (KL) 

In [2]:
import gym
import gym_electricitymarket
from drpo import DRTRPOAgent 
import time
import numpy as np

# Create Gym environment
kl_env = 'ElectricityMarketDiscreteDQN-v0'
env = gym.make(kl_env)

# Check agent class for initialization parameters and initialize agent

# When the learning rate is large, policy neural network can overflow and lead to NaNs. 
# A possible fix is to reduce lr or increase beta to lower the learning rate.

if kl_env == "ElectricityMarketDiscreteDQN-v0":
    gamma = 0.95
    lr = 1e-2
    beta = 8
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 1500
max_steps = 30

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
        
    avg_episode_reward = episode_reward/env.action_space.n        
    # add randomness for better exploration
#     if (avg_episode_reward <= 300) and (episode % 10 == 0):
#         state_adv[0] += (np.random.random()-0.5)*0.5
#         state_adv[1] += (np.random.random()-0.5)*0.5
    
#     state_adv[0] += 0.5
    
    # restart the agent if stuck
#     if (episode >= 5) and (avg_episode_reward <= 15):
#         agent = DRTRPOAgent(env, gamma, lr)   
    
    policy_loss = agent.compute_policy_loss_kl(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

Episode 0: -491.1826747201384
Episode 1: -493.98000124444064
Episode 2: -489.2814807604151
Episode 3: -488.5239604777824
Episode 4: -493.38088224166466
Episode 5: -483.53980325000003
Episode 6: -491.89671946111287
Episode 7: -480.2960591381999
Episode 8: -484.0236022166655
Episode 9: -465.12932969653144
Episode 10: -466.59759379722715
Episode 11: -464.62985525000107
Episode 12: -448.4046362486137
Episode 13: -435.8284226694431
Episode 14: -420.98920039166546
Episode 15: -406.3595188215275
Episode 16: -392.3103958437472
Episode 17: -383.3689863638906
Episode 18: -370.4175169791679
Episode 19: -349.06115023958193
Episode 20: -343.3717857326403
Episode 21: -313.0377076881955
Episode 22: -296.79838484583496
Episode 23: -294.06449244166964
Episode 24: -269.81520177986823
Episode 25: -252.65939507500448
Episode 26: -232.61235308959047
Episode 27: -219.92724155625615
Episode 28: -219.93686887500428
Episode 29: -209.60514109583912
Episode 30: -199.7972090673656
Episode 31: -201.43320306042034


Episode 264: 451.1630604167935
Episode 265: 451.1630604167935
Episode 266: 451.1630604167935
Episode 267: 451.1630604167935
Episode 268: 451.1630604167935
Episode 269: 451.1630604167935
Episode 270: 451.1630604167935
Episode 271: 451.1630604167935
Episode 272: 451.1630604167935
Episode 273: 451.1630604167935
Episode 274: 451.1630604167935
Episode 275: 451.1630604167935
Episode 276: 451.1630604167935
Episode 277: 451.1630604167935
Episode 278: 451.1630604167935
Episode 279: 451.1630604167935
Episode 280: 451.1630604167935
Episode 281: 451.1630604167935
Episode 282: 451.1630604167935
Episode 283: 451.1630604167935
Episode 284: 451.1630604167935
Episode 285: 451.1630604167935
Episode 286: 451.1630604167935
Episode 287: 451.1630604167935
Episode 288: 451.1630604167935
Episode 289: 451.1630604167935
Episode 290: 451.1630604167935
Episode 291: 451.1630604167935
Episode 292: 451.1630604167935
Episode 293: 451.1630604167935
Episode 294: 451.1630604167935
Episode 295: 451.1630604167935
Episode 

Episode 529: 451.1630604167935
Episode 530: 451.1630604167935
Episode 531: 451.1630604167935
Episode 532: 451.1630604167935
Episode 533: 451.1630604167935
Episode 534: 451.1630604167935
Episode 535: 451.1630604167935
Episode 536: 451.1630604167935
Episode 537: 451.1630604167935
Episode 538: 451.1630604167935
Episode 539: 451.1630604167935
Episode 540: 451.1630604167935
Episode 541: 451.1630604167935
Episode 542: 451.1630604167935
Episode 543: 451.1630604167935
Episode 544: 451.1630604167935
Episode 545: 451.1630604167935
Episode 546: 451.1630604167935
Episode 547: 451.13353291679346
Episode 548: 451.1630604167935
Episode 549: 451.1630604167935
Episode 550: 451.1630604167935
Episode 551: 451.1630604167935
Episode 552: 451.1630604167935
Episode 553: 451.1630604167935
Episode 554: 451.1630604167935
Episode 555: 451.1630604167935
Episode 556: 451.1630604167935
Episode 557: 451.1630604167935
Episode 558: 451.1630604167935
Episode 559: 451.1630604167935
Episode 560: 451.1630604167935
Episode

Episode 794: 451.1630604167935
Episode 795: 451.1630604167935
Episode 796: 451.1630604167935
Episode 797: 451.1630604167935
Episode 798: 451.1630604167935
Episode 799: 451.1630604167935
Episode 800: 451.1630604167935
Episode 801: 451.1630604167935
Episode 802: 451.1630604167935
Episode 803: 451.1630604167935
Episode 804: 451.1630604167935
Episode 805: 451.1630604167935
Episode 806: 451.1630604167935
Episode 807: 451.1630604167935
Episode 808: 451.1630604167935
Episode 809: 451.1630604167935
Episode 810: 451.1630604167935
Episode 811: 451.1630604167935
Episode 812: 451.1630604167935
Episode 813: 451.1630604167935
Episode 814: 451.1630604167935
Episode 815: 451.1630604167935
Episode 816: 451.1630604167935
Episode 817: 451.1630604167935
Episode 818: 451.1630604167935
Episode 819: 451.1630604167935
Episode 820: 451.1630604167935
Episode 821: 451.1630604167935
Episode 822: 451.1630604167935
Episode 823: 451.1630604167935
Episode 824: 451.1630604167935
Episode 825: 451.1630604167935
Episode 

Episode 1057: 451.1630604167935
Episode 1058: 451.1630604167935
Episode 1059: 451.1630604167935
Episode 1060: 451.1630604167935
Episode 1061: 451.1630604167935
Episode 1062: 451.1630604167935
Episode 1063: 451.1630604167935
Episode 1064: 451.1630604167935
Episode 1065: 451.1630604167935
Episode 1066: 451.1630604167935
Episode 1067: 451.1630604167935
Episode 1068: 451.1630604167935
Episode 1069: 451.1630604167935
Episode 1070: 451.1630604167935
Episode 1071: 451.1630604167935
Episode 1072: 451.1630604167935
Episode 1073: 451.1539779167935
Episode 1074: 451.1630604167935
Episode 1075: 451.1630604167935
Episode 1076: 451.1630604167935
Episode 1077: 451.1630604167935
Episode 1078: 451.1630604167935
Episode 1079: 451.1630604167935
Episode 1080: 451.1630604167935
Episode 1081: 451.1630604167935
Episode 1082: 451.1630604167935
Episode 1083: 451.1630604167935
Episode 1084: 451.1630604167935
Episode 1085: 451.1630604167935
Episode 1086: 451.1630604167935
Episode 1087: 451.1630604167935
Episode 

Episode 1313: 451.1630604167935
Episode 1314: 451.1630604167935
Episode 1315: 451.1630604167935
Episode 1316: 451.1630604167935
Episode 1317: 451.1630604167935
Episode 1318: 451.1630604167935
Episode 1319: 451.1630604167935
Episode 1320: 451.1630604167935
Episode 1321: 451.1630604167935
Episode 1322: 451.1630604167935
Episode 1323: 451.1387971417935
Episode 1324: 451.1630604167935
Episode 1325: 451.1630604167935
Episode 1326: 451.1630604167935
Episode 1327: 451.1550604167935
Episode 1328: 451.1630604167935
Episode 1329: 451.1630604167935
Episode 1330: 451.1630604167935
Episode 1331: 451.1630604167935
Episode 1332: 451.1630604167935
Episode 1333: 451.1630604167935
Episode 1334: 451.1630604167935
Episode 1335: 451.1630604167935
Episode 1336: 451.1630604167935
Episode 1337: 451.1630604167935
Episode 1338: 451.1630604167935
Episode 1339: 451.1630604167935
Episode 1340: 451.1630604167935
Episode 1341: 451.1630604167935
Episode 1342: 451.1630604167935
Episode 1343: 451.1630604167935
Episode 

In [3]:
dr_trpo_kl_rewards = episode_rewards
dr_trpo_kl_runtime = run_time

In [4]:
name = './log_files/dr_trpo_kl/' + kl_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_kl_runtime, dr_trpo_kl_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Wasserstein)

In [119]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

wass_env = "CartPole-v1"
# Create Gym environment
env = gym.make(wass_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 150
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss
    
    total_adv_diff += abs(state_adv[1] - state_adv[0])
    # larger beta, better stability; smaller beta, better exploration
    beta = total_adv_diff/episode 
    beta += np.random.random()*0.3-0.1
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    if (episode % 10 == 0) and (avg_episode_reward <= 350): 
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        
    state_adv[0] += 0.5
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= 15):
        agent = DRTRPOAgent(env, gamma, lr)
    
    policy_loss = agent.compute_policy_loss_wass(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_wass_rewards = episode_rewards
dr_trpo_wass_runtime = run_time

Episode 0: 20.0
Episode 1: 15.0
Episode 2: 13.5
Episode 3: 12.5
Episode 4: 11.0
Episode 5: 11.0
Episode 6: 15.5
Episode 7: 20.0
Episode 8: 20.5
Episode 9: 20.5
Episode 10: 40.0
Episode 11: 50.5
Episode 12: 86.5
Episode 13: 75.0
Episode 14: 62.5
Episode 15: 56.5
Episode 16: 46.5
Episode 17: 75.0
Episode 18: 46.0
Episode 19: 52.5
Episode 20: 50.0
Episode 21: 35.0
Episode 22: 26.0
Episode 23: 27.0
Episode 24: 17.5
Episode 25: 16.5
Episode 26: 14.5
Episode 27: 20.5
Episode 28: 18.0
Episode 29: 18.0
Episode 30: 19.0
Episode 31: 24.0
Episode 32: 19.0
Episode 33: 27.0
Episode 34: 32.0
Episode 35: 59.5
Episode 36: 71.5
Episode 37: 63.0
Episode 38: 105.0
Episode 39: 41.0
Episode 40: 33.5
Episode 41: 30.0
Episode 42: 26.5
Episode 43: 26.5
Episode 44: 19.0
Episode 45: 28.0
Episode 46: 27.0
Episode 47: 31.5
Episode 48: 38.0
Episode 49: 44.0
Episode 50: 47.0
Episode 51: 83.5
Episode 52: 142.5
Episode 53: 95.0
Episode 54: 76.5
Episode 55: 93.5
Episode 56: 53.0
Episode 57: 91.5
Episode 58: 74.5
Episo

In [120]:
name = './log_files/dr_trpo_wass/' + wass_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_wass_runtime, dr_trpo_wass_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')

## DRPO Agent (Sinkhorn)

In [ ]:
import gym
from drpo import DRTRPOAgent 
import time
import numpy as np

sink_env = "CartPole-v1"
# Create Gym environment
env = gym.make(sink_env)

# Check agent class for initialization parameters and initialize agent
if wass_env == "CartPole-v1":
    gamma = 0.95
    lr = 1e-2
    
agent = DRTRPOAgent(env, gamma, lr)

# Define training parameters
max_episodes = 200
max_steps = 500
total_adv_diff = 0

episode_rewards = []
run_time = []
start_time = time.time()
for episode in range(max_episodes):
    if episode == 0:
        first_state = env.reset()
    else:
        first_state = state
    state_adv = []
    total_value_loss = 0
    
    episode_reward = 0
    # loop through the first action
    for i in range(env.action_space.n):
        env.reset()
        state = first_state
        action = i
        trajectory = []
        
        for step in range(max_steps):
            if step != 0:
                action = agent.get_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectory.append((state, action, reward, next_state, done))
            episode_reward += reward  
            if done or step == max_steps-1:
                break
            state = next_state
            
        adv, value_loss = agent.compute_adv_mc(trajectory)
        state_adv.append(adv[0])
        total_value_loss += value_loss

    total_adv_diff += abs(state_adv[1] - state_adv[0])
    # larger beta, better stability; smaller beta, better exploration
    beta = total_adv_diff/episode 
    beta += np.random.random()*0.3-0.1
    
    avg_episode_reward = episode_reward/env.action_space.n
    # add randomness for better exploration
    if (episode % 10 == 0) and (avg_episode_reward <= 350): 
        state_adv[0] += (np.random.random()-0.5)*0.5
        state_adv[1] += (np.random.random()-0.5)*0.5
        
    # restart the agent if stuck
    if (episode >= 5) and (avg_episode_reward <= 10):
        agent = DRTRPOAgent(env, gamma, lr)
    
    beta = 50
    policy_loss = agent.compute_policy_loss_sinkhorn(first_state, state_adv, beta)
    agent.update(value_loss, policy_loss)
    elapse = time.time() - start_time
    run_time.append(elapse)
    
    episode_rewards.append(avg_episode_reward)
    print("Episode " + str(episode) + ": " + str(avg_episode_reward))

dr_trpo_sink_rewards = episode_rewards
dr_trpo_sink_runtime = run_time

In [ ]:
name = './log_files/dr_trpo_sink/' + sink_env + '-' + str(time.time()) + '.csv' 
out = np.column_stack((dr_trpo_sink_runtime, dr_trpo_sink_rewards))
with open(name, 'ab') as f:
    np.savetxt(f, out, delimiter=',')